In [1]:
import pandas as pd, numpy as np
import pickle, re
from datetime import datetime, timedelta

In [2]:
all_fips = sorted(set(pickle.load(open('flow.pickle', 'rb'))['sfips']))
all_fips = [x for x in all_fips if x//1000 != 72]
print(len(all_fips))

3142


In [3]:
ecbd = pickle.load(open('time_series.pickle','rb'))['ECBD'].loc[all_fips]
matrices = pickle.load(open('matrices.pickle','rb'))
cases_full = pickle.load(open('time_series.pickle','rb'))['cases_full'].loc[all_fips]
cases_added = pickle.load(open('time_series.pickle','rb'))['cases_added'].loc[all_fips]

In [4]:
t14_dates = sorted(set(ecbd).intersection(set(matrices)))

t14_cases = pd.DataFrame()

t14_recovered = pd.DataFrame()

recovered = pd.Series(ecbd[t14_dates[0]])*0
c = pd.Series(ecbd[t14_dates[0]])

alpha= 80
beta = .9


for date in t14_dates:
    p = ecbd[date]
    c = alpha*p + beta*c
    c = matrices[date] @ c
    
    t14_cases[date] = c
    recovered += (1-beta)*c
    t14_recovered[date] = recovered


In [5]:
t14_cases = t14_cases.set_index(pd.Index(all_fips))
t14_recovered = t14_recovered.set_index(pd.Index(all_fips))

In [6]:
ratios = cases_full.sum(axis=1)/(t14_cases.sum(axis=1) + t14_recovered.sum(axis=1))
ratios = ratios.fillna(ratios.mean())

In [7]:
# duplicating last mactrix
last_matrix = sorted(list(matrices))[-1]
for date in set(cases_full).difference(set(matrices)):
    if date > last_matrix:
        print(date)
        matrices[date] = matrices[last_matrix]

2020-04-27 00:00:00
2020-04-30 00:00:00
2020-04-29 00:00:00
2020-04-28 00:00:00


In [8]:
estimated_cases = t14_cases.copy()
estimated_recovered = t14_recovered.copy()

c = estimated_cases[list(t14_recovered)[-1]]
recovered = estimated_recovered[list(t14_recovered)[-1]]

alpha= 1
beta = .9

for date in sorted(set(matrices).difference(set(ecbd))):
    p = ratios*cases_added[date]
    c = alpha*p + beta*c
    c = matrices[date] @ c
    estimated_cases[date] = c
    recovered += (1-beta)*c
    estimated_recovered[date] = recovered

In [9]:
estimates = {'t14_cases':t14_cases,'t14_recovered':t14_recovered,'estimated_cases':estimated_cases,'estimated_recovered':estimated_recovered}

In [10]:
pickle.dump(estimates,open('estimates.pickle','wb'))